<a href="https://colab.research.google.com/github/raqueeb/nlp_bangla/blob/master/Chap_7_transformers_bert_Bangla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## হাতেকলমে ন্যাচারাল ল্যাঙ্গুয়েজ প্রসেসিং: শুরুর ধারণা

[৭ম অধ্যায়](https://aiwithr.github.io/nlpbook/)

In [3]:
import os
os.environ['KAGGLE_USERNAME'] = "test" # username from the json file
os.environ['KAGGLE_KEY'] = "xxxxxx"

In [4]:
!kaggle datasets files furcifer/bangla-newspaper-dataset

name                  size  creationDate         
--------------------  ----  -------------------  
data_v2/data_v2.json   2GB  2020-10-21 06:09:48  
data/data.json         4GB  2020-10-21 06:09:48  


In [5]:
!kaggle datasets download furcifer/bangla-newspaper-dataset -f data_v2/data_v2.json

 97% 430M/444M [00:05<00:00, 67.4MB/s]
100% 444M/444M [00:05<00:00, 85.9MB/s]


In [6]:
!unzip data_v2.json.zip

Archive:  data_v2.json.zip
  inflating: data_v2.json            


In [7]:
import json
with open('data_v2.json', encoding='utf-8') as f:
    datastore = json.load(f)

sentences = []
labels = []

for item in datastore:
    sentences.append(item['content'])
    labels.append(item['category'])

In [8]:
set(labels)

{'bangladesh',
 'economy',
 'education',
 'entertainment',
 'international',
 'life-style',
 'opinion',
 'sports',
 'technology'}

In [9]:
categories = list(set(labels))

In [10]:
categories

['sports',
 'economy',
 'opinion',
 'technology',
 'entertainment',
 'bangladesh',
 'education',
 'international',
 'life-style']

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
  sentences, labels, test_size=0.2, random_state=42)

In [9]:
!pip install ktrain

In [10]:
import ktrain
from ktrain import text

In [11]:
MODEL_NAME = 'sagorsarker/bangla-bert-base'
t = text.Transformer(MODEL_NAME, maxlen=500)

In [ ]:
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)

preprocessing train...
language: bn
train sequence lengths:
	mean : 268
	95percentile : 669
	99percentile : 1079


In [ ]:
model = t.get_classifier()

In [ ]:
# ট্রেনিং করে নিচ্ছি
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [ ]:
%matplotlib inline
learner.lr_find(show_plot=True, max_epochs=1)

In [ ]:
# preproc = t
# save model and Preprocessor instance after partially training
# ktrain.get_predictor(model, preproc).save('/tmp/my_predictor')

In [ ]:
# reload Predictor and extract model
# model = ktrain.load_predictor('/tmp/my_predictor').model

In [ ]:
learner.fit_onecycle(8e-5, 1)

In [ ]:
learner.validate(class_names=t.get_classes())

In [ ]:
# the one we got most wrong
learner.view_top_losses(n=1, preproc=t)

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [ ]:
# Mount your google drive in google colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
predictor.save('/content/drive/MyDrive/Colab Notebooks/saved_model/bangla_news_predictor_kt')
# save model using transformers API after partially training
learner.model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/saved_model/bangla_news_predictor')


In [ ]:
!ls -alh '/content/drive/MyDrive/Colab Notebooks/saved_model/bangla_news_predictor_kt'

In [ ]:
predictor.predict('বাংলাদেশে নদ-নদীগুলো ছড়িয়ে আছে জালের মতো। ঐতিহাসিকভাবেই এই অঞ্চলের নদ-নদীর গতি–প্রকৃতি ও প্রবাহ জটিল।')

In [ ]:
predictor.explain('বাংলাদেশে নদ-নদীগুলো ছড়িয়ে আছে জালের মতো। ঐতিহাসিকভাবেই এই অঞ্চলের নদ-নদীর গতি–প্রকৃতি ও প্রবাহ জটিল।')

In [ ]:
predictor.predict('বাংলাদেশ টেলিভিশনে গান হয়, সংস্কৃতিকে তুলে ধরতে হবে আমাদের।')

In [ ]:
predictor.predict_proba('বাংলাদেশে টেলিভিশনে গান হয়, সংস্কৃতিকে তুলে ধরতে হবে আমাদের।')

In [ ]:
predictor.predict('বাংলাদেশে ফুটবল খুব বেশি খেলা হয়, ক্রিকেট কম নয়।')

In [ ]:
!pip install git+https://github.com/amaiya/eli5@tfkeras_0_10_1

In [ ]:
predictor.explain('শিক্ষা মন্ত্রণালয়ে এমন প্রস্তাব দিয়েছে মাউশি। মাউশির কর্মকর্তারা বলছেন, মন্ত্রণালয় অনুমোদন দিলেই সবাইকে সময়সহ অন্যান্য আনুষঙ্গিক বিষয় জানিয়ে দেওয়া হবে। তখন বেসরকারি বিদ্যালয়গুলোও কীভাবে নিজস্ব ব্যবস্থায় আবেদনপত্র বিতরণ ও লটারির কাজটি করবে, তা-ও জানিয়ে দেওয়া হবে। একই সঙ্গে ঢাকার বাইরের সরকারি বিদ্যালয়গুলোর বিষয়েও দিকনির্দেশনা দেওয়া হবে।')

In [ ]:
# টেন্সরফ্লোতে মডেল সেভ করা
# model.save('/content/gdrive/My Drive/Colab Notebooks/saved_model/my_model')

In [ ]:
# save Predictor (i.e., model and Preprocessor instance) after partially training
ktrain.get_predictor(model, t).save('/content/gdrive/My Drive/Colab Notebooks/saved_model/bangla_news_predictor_kt')

In [ ]:
!ls -alh '/content/gdrive/My Drive/Colab Notebooks/saved_model/bangla_news_predictor_kt'

In [ ]:
# reload the model using transformers directly
# from transformers import *
# model = TFAutoModelForSequenceClassification.from_pretrained('/tmp/my_model')
# model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [ ]:
reloaded_predictor = ktrain.load_predictor('/content/gdrive/My Drive/Colab Notebooks/saved_model/bangla_news_predictor_kt')

In [ ]:
reloaded_predictor.predict_proba('বঙ্গবন্ধু টি–টোয়েন্টি কাপে কাল জেমকন খুলনার হয়ে খেলার ম্যাচে ৪ ওভারে ২৮ রানে ১ উইকেট নেন মাশরাফি বিন মুর্তজা।')

In [ ]:
reloaded_predictor.get_classes()